In [1]:
import warnings
warnings.filterwarnings('ignore')  # 不打印 warning 

import tensorflow as tf

# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

import numpy as np

In [4]:
def conv2d(scope, x, filter_shape, strides_x, strides_y, padding, std=0.01, biases=0.1):
    """ 高斯初始化： std=0.01, biases=0.1
    Args:
        scope: scope name of this layer.
        x: 4-D inputs. [batch_size, in_height, in_width, in_channels]
        filter_shape: A list of ints.[filter_height, filter_width, in_channels, out_channels]
        strides: A list of ints. 1-D tensor of length 4. The stride of the sliding window for each dimension of input.
        padding: A string from: "SAME", "VALID". The type of padding algorithm to use.
    Returns:
        h_conv:  A 4-D tensor. [batch_size, out_height, out_width, out_channels].
        if padding is 'SAME', then out_height==in_height.
        else, out_height = in_height - filter_height + 1.
        the same for out_width.
    """
    assert padding in ['SAME', 'VALID']
    strides = [1, strides_x, strides_y, 1]
    with tf.variable_scope(scope):
        W_conv = tf.Variable(tf.truncated_normal(filter_shape, dtype=tf.float32, stddev=std), name='weights')
        b_conv = tf.Variable(tf.constant(biases, shape=[filter_shape[-1]], dtype=tf.float32), name='biases')
        h_conv = tf.nn.conv2d(x, W_conv, strides=strides, padding=padding)
        h_conv_relu = tf.nn.relu(h_conv + b_conv)
    return h_conv_relu

def max_pooling(scope, x, k_height, k_width, strides_x, strides_y, padding='SAME'):
    """max pooling layer."""
    with tf.variable_scope(scope):
        ksize = [1, k_height, k_width, 1]
        strides = [1, strides_x, strides_y, 1]
        h_pool = tf.nn.max_pool(x, ksize, strides, padding)
    return h_pool



inputs = tf.placeholder(dtype=tf.float32, shape=[None, 256, 256, 3], name='inputs')
print(inputs)

# 1st Layer: Conv (w ReLu) -> Lrn -> Pool
conv1 = conv2d('conv1', inputs, [11, 11, 3, 96], 4, 4, 'VALID', biases=0.0)
pool1 = max_pooling('pool1', conv1, 3, 3, 2, 2, 'VALID')
print('conv1', conv1)
print('pool1', pool1)

# 2nd Layer: Conv (w ReLu)  -> Lrn -> Pool with 2 groups
conv2 = conv2d('conv2', pool1, [5, 5, 96, 256], 1, 1, 'SAME')
pool2 = max_pooling('pool2', conv2, 3, 3, 2, 2, 'VALID')
print('conv2', conv2d)
print('pool2', pool2)

# 3rd Layer: Conv (w ReLu)
conv3 = conv2d('conv3', pool2, [3, 3, 256, 384], 1, 1, 'SAME')
print('conv3', conv3)

# 4th Layer: Conv (w ReLu)
conv4 = conv2d('conv4', conv3, [3, 3, 384, 384], 1, 1, 'SAME')
print('conv4', conv4)

# 5th Layer: Conv (w ReLu) -> Pool splitted into two groups
conv5 = conv2d('conv5', conv4, [3, 3, 384, 256], 1, 1, 'SAME')
pool5 = max_pooling('pool5', conv5, 3, 3, 2, 2, 'VALID')
print('conv5', conv5)
print('pool5', pool5)


# # 6th Layer: Flatten -> FC (w ReLu) -> Dropout
# flattened = tf.reshape(pool5, [-1, 6 * 6 * 256])
# fc6 = fc('fc6', flattened, 6 * 6 * 256, 4096, std=0.005, biases=0.1, use_relu=True)
# dropout6 = dropout(fc6, keep_prob)

# # 7th Layer: FC (w ReLu) -> Dropout
# fc7 = fc('fc7', dropout6, 4096, 4096, std=0.005, biases=0.1, use_relu=True)
# dropout7 = dropout(fc7, keep_prob)

# # 8th Layer: FC and return unscaled activations
# fc8 = fc('fc8', dropout7, 4096, n_class, std=0.01, biases=0.0, use_relu=False)

Tensor("inputs_2:0", shape=(?, 256, 256, 3), dtype=float32)
conv1 Tensor("conv1/Relu:0", shape=(?, 62, 62, 96), dtype=float32)
pool1 Tensor("pool1/MaxPool:0", shape=(?, 30, 30, 96), dtype=float32)
conv2 <function conv2d at 0x7f50ac498950>
pool2 Tensor("pool2/MaxPool:0", shape=(?, 14, 14, 256), dtype=float32)
conv3 Tensor("conv3/Relu:0", shape=(?, 14, 14, 384), dtype=float32)
conv4 Tensor("conv4/Relu:0", shape=(?, 14, 14, 384), dtype=float32)
conv5 Tensor("conv5/Relu:0", shape=(?, 14, 14, 256), dtype=float32)
pool5 Tensor("pool5/MaxPool:0", shape=(?, 6, 6, 256), dtype=float32)


In [5]:
6 * 6 * 256

9216